In [1]:
import pandas as pd
import logging
from datetime import datetime
import csv
from collections import defaultdict

In [2]:
# set up logging file
# Step 1: Configure the logging
logging.basicConfig(filename='../logs/gene_length.log',  # Log file path
                    filemode='a',  # Append mode (use 'w' for overwrite mode)
                    level=logging.INFO,  # Log level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
                    format='%(asctime)s - %(levelname)s - %(message)s',  # Log message format
                    datefmt='%Y-%m-%d %H:%M:%S')  # Date format
# Step 2: Write log messages
# Generate a timestamp
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
logging.info(f"Starting gene length calculation: {timestamp}")  # Example of logging an informational message

# Process
Normally I would use a gff parser, but I think this may be too heavy weight for what I need. 

1. Read data/references/GCF_000001405.40_GRCh38.p14_genomic.gff
2. Keep only 'gene' lines.
3. Calculate length: $5-$4+1
4. Gene gene id from column 9
5. Get gene symbol from column 9

In [3]:
file='../data/references/GCF_000001405.40_GRCh38.p14_genomic.gff'

genes = []

with open(file, 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        if len(row) > 2 and row[2] == 'gene':
            genes.append(row)

logging.info(f"Filtered gene lines: {len(genes)}")

In [5]:
gene_info = []

for gene in genes:
    start = int(gene[3])
    end = int(gene[4])
    length = end - start + 1
    
    attributes = gene[8].split(';')
    name = None
    gene_id = None
    
    for attribute in attributes:
        if attribute.startswith('Name='):
            name = attribute.split('=')[1]
        elif attribute.startswith('Dbxref=GeneID:'):
            gene_id = attribute.split(':')[1].split(',')[0]
    
    gene_info.append({'length': length, 'name': name, 'gene_id': gene_id})

logging.info(f"Processed gene information: {len(gene_info)}")
print(gene_info[:5])  # Print first 5 entries for verification

[{'length': 68, 'name': 'MIR6859-1', 'gene_id': '102466751'}, {'length': 5645, 'name': 'MIR1302-2HG', 'gene_id': '107985730'}, {'length': 138, 'name': 'MIR1302-2', 'gene_id': '100302278'}, {'length': 1471, 'name': 'FAM138A', 'gene_id': '645520'}, {'length': 6167, 'name': 'OR4F5', 'gene_id': '79501'}]


In [6]:
output_file = '../data/processed/gene_info.csv'

# Define the column order
columns = ['gene_id', 'name', 'length']

# Write the gene_info to a CSV file
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=columns)
    writer.writeheader()
    writer.writerows(gene_info)

logging.info(f"Gene information written to {output_file}")
print(f"Gene information written to {output_file}")

Gene information written to ../data/processed/gene_info.csv
